# TNT Cates: Creep Compliance
> **Handbook:** This notebook demonstrates the TNT Cates model. For complete mathematical derivations and theoretical background, see [TNT Cates Documentation](../../docs/source/models/tnt/tnt_cates.rst).


**Estimated Time:** 4-6 minutes

## Protocol: Creep in Living Polymers

**Creep** in living polymers shows **delayed flow** with a characteristic lag time related to the geometric mean relaxation time $\tau_d$.

### Physical Context for Living Polymers

When a constant stress is applied to wormlike micelles:

1. **Immediate elastic response**: Chains stretch (compliance $J_e = 1/G$)
2. **Transient regime** (0 < t < $\tau_d$): Two-timescale relaxation (fast scission + slow reptation)
3. **Steady flow** (t > $\tau_d$): Viscous creep with shear rate $\dot{\gamma} = \sigma_0 / \eta(\sigma_0)$

The Cates model predicts **stress-dependent viscosity** from:
- **Stress-induced scission**: Higher stress → faster chain breaking → shorter chains → lower viscosity
- **Shear alignment**: Chains orient along flow, reducing entanglements

This leads to **non-Newtonian creep** where the long-time creep rate depends on the applied stress magnitude.

> **Cates Creep Physics**  
> For stress-dependent viscosity and delayed flow analysis, see:  
> [../../docs/source/models/tnt/tnt_cates.rst](../../docs/source/models/tnt/tnt_cates.rst) — Section on "Creep"

---

## Learning Objectives

1. Understand delayed flow and stress-dependent viscosity in living polymers
2. Fit TNT Cates model to creep compliance data
3. Extract $\tau_d$ from transient regime duration
4. Analyze stress-dependent steady-state creep rate
5. Compare Cates (nonlinear creep) to Single-Mode (linear creep)
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Cates fundamentals (Notebook 07)
- Understanding of creep (TNT Single-Mode NB04)
- Familiarity with nonlinear rheology

## Runtime Estimate

- NLSQ fitting: ~5-10 seconds
- Bayesian inference (demo): ~60-120 seconds
- Total: ~5-8 minutes

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumulative_trapezoid

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_creep,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Cates Model for Creep

The Cates model predicts creep compliance following step stress:

**Creep compliance (UCM-like):**
In the fast-breaking limit:
$$J(t) = \frac{1}{G_0} + \frac{t}{G_0 \tau_d} + \frac{t}{\eta_s}$$

where:
- $1/G_0$: Elastic compliance (instantaneous)
- $t/(G_0 \tau_d)$: Viscoelastic creep (linear in time)
- $t/\eta_s$: Viscous flow (steady state)

**Effective relaxation time:**
$$\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$$

**Physical interpretation:**
- Faster breaking → shorter $\tau_d$ → faster creep
- Slope of $J(t)$ at intermediate times gives $1/(G_0 \tau_d)$

## Load Creep Data and Convert to Strain

In [ ]:
time_data, shear_rate, _, stress_applied = load_ml_ikh_creep(stress_pair_index=0)

strain = cumulative_trapezoid(shear_rate, time_data, initial=0)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Applied stress: {stress_applied:.2f} Pa")
print(f"Strain range: {strain.min():.2e} to {strain.max():.2e}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, strain, 'o', label='Strain data', markersize=4)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain γ', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Creep Strain', fontsize=14)

compliance = strain / stress_applied
ax2.loglog(time_data, compliance, 'o', label='J(t)', markersize=4)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_title('Creep Compliance', fontsize=14)

plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

# Smart initialization for creep mode
# Estimate G_0 from compliance at short times
# eta_s must be non-zero for numerical stability in creep ODE
strain_early = np.mean(strain[1:5]) if len(strain) > 5 else strain[1]
G_0_init = max(stress_applied / strain_early, 10.0) if strain_early > 0 else 100.0

# Estimate tau_d from characteristic time
t_char = time_data[len(time_data)//4]
tau_d_init = max(t_char, 0.1)

# For Cates model: tau_d = sqrt(tau_rep * tau_break)
# Use a ratio approximation
tau_rep_init = tau_d_init / np.sqrt(0.1)  # Assume tau_break/tau_rep ~ 0.1
tau_break_init = 0.1 * tau_rep_init

# Set eta_s for numerical stability
eta_0_init = G_0_init * tau_d_init
eta_s_init = 0.01 * eta_0_init

print(f"Initial estimates from data:")
print(f"  G_0_init = {G_0_init:.2f} Pa")
print(f"  tau_rep_init = {tau_rep_init:.4f} s")
print(f"  tau_break_init = {tau_break_init:.4f} s")
print(f"  eta_s_init = {eta_s_init:.4f} Pa·s")

model.parameters.set_value('G_0', G_0_init)
model.parameters.set_value('tau_rep', tau_rep_init)
model.parameters.set_value('tau_break', tau_break_init)
model.parameters.set_value('eta_s', eta_s_init)

start_time = time.time()
model.fit(time_data, strain, test_mode='creep', sigma_applied=stress_applied, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
strain_pred_fit = model.predict(time_data, test_mode='creep', sigma_applied=stress_applied)
quality = compute_fit_quality(strain, strain_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(strain, model.predict(time_data, test_mode='creep', sigma_applied=stress_applied))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, strain, model, test_mode="creep",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Strain $\gamma$',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})',
    sigma_applied=stress_applied
)
plt.close("all")

## Physical Analysis: Extract tau_d from Compliance

In [ ]:
# Generate fine grid for smooth predictions
time_pred = np.linspace(time_data.min(), time_data.max(), 200)
strain_pred = model.predict(time_pred, test_mode='creep', sigma_applied=stress_applied)

tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

J_elastic = 1.0 / nlsq_params['G_0']
J_slope = 1.0 / (nlsq_params['G_0'] * tau_d)

compliance_pred = strain_pred / stress_applied

mask = (time_pred > tau_d/10) & (time_pred < tau_d*10)
if jnp.sum(mask) > 10:
    log_t = jnp.log(time_pred[mask])
    log_J = jnp.log(compliance_pred[mask])
    slope_fit = jnp.polyfit(log_t, log_J, 1)[0]
else:
    slope_fit = 0.0

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")
print(f"\nCompliance Components:")
print(f"  Elastic compliance (1/G_0): {J_elastic:.4e} Pa^-1")
print(f"  Viscoelastic slope (1/(G_0*tau_d)): {J_slope:.4e} Pa^-1 s^-1")
print(f"  Log-log slope near tau_d: {slope_fit:.2f} (expected ~1 for linear creep)")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: UCM-like creep behavior")
else:
    print(f"\n  → Full Cates dynamics: May deviate from simple UCM")

## Visualize Compliance Components

In [ ]:
J_elastic_line = J_elastic * np.ones_like(time_pred)
J_viscous_line = J_elastic + time_pred * J_slope

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, compliance, 'o', label='Data', markersize=4, zorder=3)
ax.loglog(time_pred, compliance_pred, '-', linewidth=2, label='TNTCates fit', zorder=2)
ax.loglog(time_pred, J_elastic_line, '--', linewidth=1.5, label='Elastic (1/G₀)', zorder=1)
ax.loglog(time_pred, J_viscous_line, ':', linewidth=1.5, label='Elastic + Viscous', zorder=1)
ax.axvline(tau_d, color='gray', linestyle='-.', alpha=0.7, label=f'τ_d = {tau_d:.2e}s')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa$^{-1}$)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Compliance Components', fontsize=14)
plt.close("all")
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        time_data if 'time_data' in dir() else t_data,
        stress if 'stress' in dir() else strain,
        test_mode='creep',
        sigma_applied=stress_applied,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data,
        strain,
        model, bayesian_result, test_mode="creep",
        param_names=param_names, log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Strain $\\gamma$', sigma_applied=stress_applied
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']
J_slope_posterior = 1.0 / (posterior['G_0'] * tau_d_posterior)

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")
print(f"  Viscoelastic slope: {np.mean(J_slope_posterior):.4e} ± {np.std(J_slope_posterior):.4e} Pa^-1 s^-1")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(J_slope_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(J_slope_posterior), color='r', linestyle='--', label='Mean')
ax2.set_xlabel(r'$1/(G_0 \tau_d)$ (Pa$^{-1}$ s$^{-1}$)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Viscoelastic Slope Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "creep", param_names)

## Key Takeaways

1. **UCM-like creep** in Cates model: $J(t) = 1/G_0 + t/(G_0 \tau_d)$
2. **Effective relaxation time** $\tau_d$ governs viscoelastic compliance slope
3. **Linear creep region** (around $\tau_d$) provides direct measurement of $\tau_d$
4. **Compliance decomposition** separates elastic, viscoelastic, and viscous contributions
5. **Bayesian inference** quantifies uncertainty in compliance parameters from creep data

**Next steps:** Compare $\tau_d$ from creep with values from other protocols (relaxation, SAOS) to verify consistency.

## Next Steps

- **Notebook 11**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
